In [ ]:
import numpy as np
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
import loss_funcs
from evaluate import eval_funcs
from algs import *

from utils.orient import OrientUtils
from evaluate.evaluator import Evaluator
from utils.visualize import *

from evaluate.dataset import Dataset

import mealpy
import cv2 as cv

plotter = SearchPlotter(update_freq=100, history=None, alpha=0.5)

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.3), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/mouse/world.xml", cam_config, simulation_time=0)
sim_viewer = ViewSampler("data/mouse/world_sim.xml", cam_config)

init_location = (0, 1.3, 0.3)
random_orientations = OrientUtils.generate_random(20)
eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]

ds = Dataset.create_random(init_location, 50)
ds.visualize(sim_viewer, depth=True, batch_size=20)

loss_func = loss_funcs.WeightedSum(loss_funcs.IOU(), loss_funcs.RMSE())
alg_config = MealAlgorithm.Config(time_limit=50)

eval_func = eval_funcs.XorDiff(0.1)

evaluator = Evaluator(world_viewer, sim_viewer, eval_func=eval_func)
evaluator.register_callback(lambda x: plotter.reset())

algorithms: list[MealAlgorithm] = [
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.OriginalSA()),  # classic
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.SwarmSA(step_size=np.pi / 100)),  # pretty bad, classic
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.OriginalPSO()),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.HPSO_TVAC()),  # very good
    MealAlgorithm(
        sim_viewer, loss_func, mealpy.music_based.HS.OriginalHS(c_r=0.5, pa_r=0.25)
    ),  # interesting but pretty bad
    MealAlgorithm(sim_viewer, loss_func, mealpy.system_based.GCO.DevGCO()),  # decent
    MealAlgorithm(sim_viewer, loss_func, mealpy.QSA.OriginalQSA()),  # seems pretty good
    MealAlgorithm(sim_viewer, loss_func, mealpy.BRO.DevBRO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.CGO.OriginalCGO()),  # weird algorithm
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.CircleSA.OriginalCircleSA(c_factor=0.8)),  # interesting
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.HC.OriginalHC()),  # classic, pretty good
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.SCA.DevSCA()),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.EP.LevyEP(bout_size=0.15)),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.DE.OriginalDE()),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.FPA.OriginalFPA()),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.GA.EliteMultiGA()),  # decent
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.MA.OriginalMA()),  # very interesting
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.CA.OriginalCA()),  # interesting
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.FBIO.DevFBIO()),  # very very good
    MealAlgorithm(
        sim_viewer, loss_func, mealpy.human_based.ICA.OriginalICA(revolution_prob=0.4, empire_count=7)
    ),  # very good and cool
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.CDO.OriginalCDO()),  # legendary name
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EFO.DevEFO()),  # very good, fast convergance
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EO.ModifiedEO()),  # promosing
]

algorithms: list[MealAlgorithm] = [
    MealAlgorithm(sim_viewer, loss_func, mealpy.ACOR.OriginalACOR()),  # interesting but pretty bad
]

for alg in algorithms:
    alg.register_callback(plotter.add_data)
    eval_losses = evaluator.evaluate(alg, alg_config, eval_positions)
    print(f"{alg.get_name()}: {eval_losses}")

In [ ]:
img, _ = sim_viewer.get_view(eval_positions[0], allow_simulation=False)
cv.imshow("image", img)
cv.waitKey(0)

In [ ]:
world_viewer.close()
sim_viewer.close()
plotter.close()